In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

from scipy import stats
from scipy.stats import norm, skew #for some statistics


from sklearn.preprocessing import StandardScaler # Used for scaling of data
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


Visualization Functions For General Purpose

In [2]:
# Outliers
def see_outlier_plot_comparison(train_with_outliers,train):
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(12, 6),constrained_layout=True)
    plt.suptitle('Before and After Outlier Deletion',fontsize=14)

    ax1.scatter(x = train_with_outliers['GrLivArea'], y = train_with_outliers['SalePrice'])
    ax1.set_ylabel('SalePrice', fontsize=13)
    ax1.set_xlabel('GrLivArea', fontsize=13)

    ax2.scatter(x = train['GrLivArea'], y = train['SalePrice'])
    ax2.set_ylabel('SalePrice', fontsize=13)
    ax2.set_xlabel('GrLivArea', fontsize=13)

    plt.show()
    
# Corrplot
def dothecorrplot(corrmat):
    corr = corrmat
    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=np.bool))
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    # Set up the matplotlib figure
    f = plt.subplots(figsize=(15, 15),dpi=300)

    ax = sns.heatmap(corr, mask = mask, cmap=cmap, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .8})

    sns.set(style = 'white', font_scale=1) 

    # Fix top bottom half
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    
# Missing Data Graph   
def missing_data_graph(all_data_na):
    f, ax = plt.subplots(figsize=(15, 12))
    plt.xticks(rotation='90')
    sns.barplot(x=all_data_na.index, y=all_data_na)
    plt.xlabel('Features', fontsize=15)
    plt.ylabel('Percent of missing values', fontsize=15)
    plt.title('Percent missing data by feature', fontsize=15)

## Import Data. Delete Outliers. Feature Correlation.

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

In [4]:
#Deleting outliers
train_with_outliers = train.copy()
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

#see_outlier_plot_comparison(train_with_outliers,train)

In [5]:
# most correlated features
corrmat = train.corr()
top_corr_features = corrmat.index[abs(corrmat["SalePrice"])>0.5]

#dothecorrplot(corrmat)

## Skewness

In [6]:
# Check 
def check_skewness(col):
    sns.distplot(train[col] , fit=norm);
    fig = plt.figure()
    res = stats.probplot(train[col], plot=plt)
    # Get the fitted parameters used by the function
    (mu, sigma) = norm.fit(train[col])
    print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
    
#check_skewness('SalePrice')

In [7]:
# Use log1p which applies log(1+x) to all elements of the column to fix skewness
train["SalePrice"] = np.log1p(train["SalePrice"])

#check_skewness('SalePrice')

## Fixing Features

In [8]:
# Concatenate train and test values.
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2917, 79)


In [9]:
# Missing Data
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})

# missing_data_graph(all_data_na)

### Handle Missing Data

Based on Description

In [10]:
feature_nans_to_none=["PoolQC","MiscFeature","Alley","Fence","FireplaceQu",'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','MSSubClass']
for col in feature_nans_to_none:
    all_data[col] = all_data[col].fillna('None')

In [11]:
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

In [12]:
abc = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','GarageYrBlt', 'GarageArea', 'GarageCars']
#all_data.groupby('GarageType')[abc].count()

In [13]:
# Since NaN means no garage
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
# Most likely zero, no bsmt
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
# Most likely no bsmt
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
# Most likely no masonry
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

In [14]:
#all_data['MSZoning'].value_counts()
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
#all_data['Utilities'].value_counts()
all_data = all_data.drop(['Utilities'], axis=1)

In [15]:
# NA means typical
all_data["Functional"] = all_data["Functional"].fillna("Typ")

In [16]:
# categorical values : its better to replace nan with the most used keyword.
mode_col = ['Electrical','KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType']
for col in mode_col:
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

In [17]:
#Check remaining missing values if any 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

,Missing Ratio


Numerical Features that are Categorical

In [18]:
#all_data['OverallCond'].value_counts() # example

#MSSubClass=The building class
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [19]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (2917, 78)


In [20]:
# Adding total sqfootage feature 
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [21]:
# Check skewed features
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
#skewness.head(10)

Box Cox Transformation and Data Separation

In [22]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)

There are 59 skewed numerical features to Box Cox transform


In [23]:
all_data_copy = all_data.copy()
all_data = pd.get_dummies(all_data)
all_data.shape

(2917, 220)

In [24]:
train = all_data[:ntrain]
test = all_data[ntrain:]
# y_train
train.shape

(1458, 220)

# Modelling

Deep Learning: All Features

In [25]:
seed = 7
np.random.seed(seed)
# split into 67% for train and 33% for test
X_tr, X_test, y_tr, y_test = train_test_split(train, np.expm1(y_train), test_size=0.33, random_state=seed)

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(120, input_dim=X_tr.shape[1], activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(optimizer ='adam', loss = 'mean_squared_logarithmic_error', 
              metrics =[metrics.msle])
    return model

In [26]:
model_dl_1 = create_model()
history = model_dl_1.fit(X_tr, y_tr, validation_data=(X_test,y_test), epochs=387, batch_size=32)
from IPython.display import clear_output
clear_output()  # clears consoke output

In [27]:
 def plot_dl_graphs():
    # summarize history for accuracy
    plt.plot(history.history['mean_squared_logarithmic_error'][20:])
    plt.plot(history.history['val_mean_squared_logarithmic_error'][20:])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'][20:])
    plt.plot(history.history['val_loss'][20:])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
#plot_dl_graphs()

In [28]:
#prediction = np.expm1(model.predict(X_test))
prediction = model_dl_1.predict(test)

In [29]:
id_col_df = pd.read_csv('test.csv')
id_col = id_col_df['Id'].values.tolist()
submission = pd.DataFrame()
submission['Id'] = id_col
submission['SalePrice'] = prediction
submission.to_csv('submission2.csv', index=False)

Moar Models : Lasso, Ridge, ElasticNet, Gradiend Boosting

In [30]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

Cross Validation

In [31]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

Modelling: 

In [ ]:
# Ridge 
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

# Lasso 
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
score = rmsle_cv(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

# ElasticNet
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

# Gradient Boosting
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Kernel Ridge score: 0.1153 (0.0075)

Lasso score: 0.1115 (0.0074)

ElasticNet score: 0.1116 (0.0074)



In [ ]:
# Simple Model Combination
LassoMd = lasso.fit(train.values,y_train)
ENetMd = ENet.fit(train.values,y_train)
KRRMd = KRR.fit(train.values,y_train)
GBoostMd = GBoost.fit(train.values,y_train)

semifinalMd = (np.expm1(LassoMd.predict(test.values)) + np.expm1(ENetMd.predict(test.values)) + np.expm1(KRRMd.predict(test.values)) + np.expm1(GBoostMd.predict(test.values)) ) / 4
semifinalMd

# NEXT LEVEL 

Deep Model With 5 cols of models and 10 cols of most important features 

In [ ]:
# Create new dataset from model prediction on train data
model_prices_pre = [np.expm1(LassoMd.predict(all_data.values)) , np.expm1(ENetMd.predict(all_data.values)) , np.expm1(KRRMd.predict(all_data.values)), np.expm1(GBoostMd.predict(all_data.values)) ]
next_level_df = pd.DataFrame(model_prices_pre).transpose()
next_level_df['FirstDlModel']=model_dl_1.predict(all_data)

# Selection of Features
for feature in top_corr_features[:10]:
    #print(train[feature])
    next_level_df[feature] = all_data[feature]

#next_level_df

Split train test sets and do the model

In [ ]:
# Sets
train_next = next_level_df[:ntrain]
test_next = next_level_df[ntrain:]
# y_train

# Modelling
seed = 7
np.random.seed(seed)
# split into 67% for train and 33% for test
X_tr, X_test, y_tr, y_test = train_test_split(train_next, np.expm1(y_train), test_size=0.4, random_state=seed)

def create_model_ff():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=X_tr.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(optimizer ='adam', loss = 'mean_squared_logarithmic_error', 
              metrics =[metrics.msle])
    return model

In [ ]:
model_ff = create_model_ff()
history3 = model_ff.fit(X_tr, y_tr, validation_data=(X_test,y_test), epochs=100, batch_size=32)
from IPython.display import clear_output
clear_output()  # clears consoke output

In [ ]:
def plot_dl_graphsz():
    # summarize history for accuracy
    plt.plot(history3.history['mean_squared_logarithmic_error'][2:])
    plt.plot(history3.history['val_mean_squared_logarithmic_error'][2:])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history3.history['loss'][2:])
    plt.plot(history3.history['val_loss'][2:])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
#plot_dl_graphsz()

In [ ]:
predictionZ = model_ff.predict(test_next)

In [ ]:
id_col_df = pd.read_csv('test.csv')
id_col = id_col_df['Id'].values.tolist()
submission = pd.DataFrame()
submission['Id'] = id_col
submission['SalePrice'] = predictionZ

submission.to_csv('submission3.csv', index=False)

SUM OF ALL


In [ ]:
result_prices_pre = [np.expm1(LassoMd.predict(test.values)) , np.expm1(ENetMd.predict(test.values)) , np.expm1(KRRMd.predict(test.values)), np.expm1(GBoostMd.predict(test.values)) ]
result_pandas = pd.DataFrame(result_prices_pre).transpose()
result_pandas['dl_ff']=model_ff.predict(test_next)
result_pandas['NextNextLevelFinalMean']=result_pandas.mean(axis=1)
result_pandas['previus_mean']=semifinalMd

#result_pandas.head()

In [ ]:
id_col_df = pd.read_csv('test.csv')
id_col = id_col_df['Id'].values.tolist()
submission = pd.DataFrame()
submission['Id'] = id_col
submission['SalePrice'] = result_pandas['NextNextLevelFinalMean'].values

submission.to_csv('submission4.csv', index=False)